In [67]:
import pandas as pd
import os
from openpyxl import load_workbook

In [68]:
# Define the path to the Excel file
folder_path = "../BoS (Business Objects) Raw Data Reports - Deidentified"
file_name = "143 RAW Client Data Export v3_EE Workflow.xlsx"
file_path = os.path.join(folder_path, file_name)

In [69]:
# Get the program from the file name
program_name = file_name.split(" ")[0]
print(f"We are looking at Program {program_name} data")

We are looking at Program 143 data


In [70]:
df_disability_ent = pd.read_excel(file_path, sheet_name="DISABILITY ENT")
df_disability_ent.head()

,EE Provider ID,Disability Determination (Entry),Disability Type (Entry),"If Yes, Expected to be of long-continued and indefinite duration and substantially impairs ability to live independently(10678)",Disability Start Date (Entry),End Date(874),Provider (417-provider),Date Added (417-date_added)
0,"Community Housing Network, Inc. - Grafton SHU ...",No (HUD),Alcohol Use Disorder (HUD),NaN,2015-08-28 12:00:00.779,NaT,HARA - Oakland County - ESG Screening(8319),2015-08-28 09:15:36.230
1,"Community Housing Network, Inc. - Grafton SHU ...",No (HUD),Both Alcohol and Drug Use Disorder (HUD),NaN,2015-08-28 12:00:00.784,NaT,HARA - Oakland County - ESG Screening(8319),2015-08-28 09:15:36.426
2,"Community Housing Network, Inc. - Grafton SHU ...",No (HUD),Developmental (HUD),NaN,2015-08-28 12:00:00.853,NaT,HARA - Oakland County - ESG Screening(8319),2015-08-28 09:15:36.456
3,"Community Housing Network, Inc. - Grafton SHU ...",No (HUD),Drug Use Disorder (HUD),NaN,2015-08-28 12:00:00.857,NaT,HARA - Oakland County - ESG Screening(8319),2015-08-28 09:15:36.458
4,"Community Housing Network, Inc. - Grafton SHU ...",No (HUD),HIV/AIDS (HUD),NaN,2015-08-28 12:00:00.858,NaT,HARA - Oakland County - ESG Screening(8319),2015-08-28 09:15:36.444


In [71]:
# Count the number of 'Yes' and 'No' in the 'Disability Determination (Entry)' column
entry_counts = df_disability_ent['Disability Determination (Entry)'].value_counts(dropna=True).reset_index()
entry_counts

,Disability Determination (Entry),count
0,No (HUD),6156
1,Yes (HUD),829
2,Client doesn't know (HUD),13
3,Data not collected (HUD),2


In [72]:
entry_counts.columns = ['Disability Status', 'Count']
entry_counts['Assessment Stage'] = 'Entry'
entry_counts['Program'] = program_name
entry_counts

,Disability Status,Count,Assessment Stage,Program
0,No (HUD),6156,Entry,143
1,Yes (HUD),829,Entry,143
2,Client doesn't know (HUD),13,Entry,143
3,Data not collected (HUD),2,Entry,143


In [73]:
# Process Exit Tab
# Similar steps
df_disability_exit = pd.read_excel(file_path, sheet_name="DISABILITY EXIT")
exit_counts = df_disability_exit['Disability Determination (Exit)'].value_counts(dropna=True).reset_index()
exit_counts.columns = ['Disability Status', 'Count']
exit_counts['Assessment Stage'] = 'Exit'
exit_counts['Program'] = program_name
exit_counts

,Disability Status,Count,Assessment Stage,Program
0,No (HUD),6712,Exit,143
1,Yes (HUD),846,Exit,143
2,Client doesn't know (HUD),14,Exit,143
3,Data not collected (HUD),3,Exit,143


In [74]:
# Join Entry and Exit vertically
combined_counts = pd.concat([entry_counts, exit_counts], ignore_index=True)
combined_counts 

,Disability Status,Count,Assessment Stage,Program
0,No (HUD),6156,Entry,143
1,Yes (HUD),829,Entry,143
2,Client doesn't know (HUD),13,Entry,143
3,Data not collected (HUD),2,Entry,143
4,No (HUD),6712,Exit,143
5,Yes (HUD),846,Exit,143
6,Client doesn't know (HUD),14,Exit,143
7,Data not collected (HUD),3,Exit,143


In [76]:
# Output Excel file path
output_file = "disability_summary.xlsx"
sheet_name = "Disability_Counts"

try:
    # If file exists, append (and replace sheet if it already exists)
    with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        combined_counts.to_excel(writer, sheet_name=sheet_name, index=False)
except FileNotFoundError:
    # If file doesn't exist, create new one
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        combined_counts.to_excel(writer, sheet_name=sheet_name, index=False)

In [77]:
combined_counts.to_csv("test.csv")